In [1]:
pip install pykrige

     |████████████████████████████████| 993kB 5.1MB/s 


In [2]:
from pykrige.ok import OrdinaryKriging
import pandas as pd
import numpy as np
from google.colab import files
import traceback

In [3]:
station_df=pd.read_excel('https://drive.google.com/uc?id=1DiAL0JzQQgSrUwXneznbe8OgDfqKKuKH', header=0)

In [4]:
station_df.head()

,Station_Number,Station_Name,Actual_open,Open,Close,LAT,LON,Height_m
0,1006,WYNDHAM AERO,01/1951,1951-01-01,2021-01-31,-15.5100,128.1503,3.8
1,1007,TROUGHTON ISLAND,09/1956,1956-09-01,2021-01-31,-13.7542,126.1485,6.0
2,1009,KURI BAY,08/1961,1961-08-01,2012-09-01,-15.4875,124.5222,12.0
3,1013,WYNDHAM,01/1968,1968-01-01,2021-01-31,-15.4869,128.1236,11.0
4,1018,MOUNT ELIZABETH,01/1973,1973-01-01,2021-01-31,-16.4181,126.1025,546.0


In [5]:
station_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186 entries, 0 to 185
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Station_Number  186 non-null    int64         
 1   Station_Name    186 non-null    object        
 2   Actual_open     186 non-null    object        
 3   Open            186 non-null    datetime64[ns]
 4   Close           186 non-null    datetime64[ns]
 5   LAT             186 non-null    float64       
 6   LON             186 non-null    float64       
 7   Height_m        186 non-null    float64       
dtypes: datetime64[ns](2), float64(3), int64(1), object(2)
memory usage: 11.8+ KB


In [6]:
station_df=station_df.rename(columns={"Station_Number": "station_number"})

In [7]:
station_df.head()

,station_number,Station_Name,Actual_open,Open,Close,LAT,LON,Height_m
0,1006,WYNDHAM AERO,01/1951,1951-01-01,2021-01-31,-15.5100,128.1503,3.8
1,1007,TROUGHTON ISLAND,09/1956,1956-09-01,2021-01-31,-13.7542,126.1485,6.0
2,1009,KURI BAY,08/1961,1961-08-01,2012-09-01,-15.4875,124.5222,12.0
3,1013,WYNDHAM,01/1968,1968-01-01,2021-01-31,-15.4869,128.1236,11.0
4,1018,MOUNT ELIZABETH,01/1973,1973-01-01,2021-01-31,-16.4181,126.1025,546.0


In [8]:
data_df=pd.read_csv('http://calwildfire.thesofttools.com/WAWeatherStationData.csv', header=0)

In [9]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1095213 entries, 0 to 1095212
Data columns (total 9 columns):
 #   Column                        Non-Null Count    Dtype 
---  ------                        --------------    ----- 
 0   station_number                1095213 non-null  int64 
 1   date                          1095213 non-null  object
 2   station_mean_temperature_c    1095213 non-null  object
 3   station_air_temperature_c     1095213 non-null  object
 4   Station_TDMEAN_c              1095213 non-null  object
 5   Maximum_Sustained_Wind_Speed  1095213 non-null  object
 6   Mean_Wind_Speed               1095213 non-null  object
 7   Mean_Sea_Level_Pressure       1095213 non-null  object
 8   Mean_Station_Pressure         1095213 non-null  object
dtypes: int64(1), object(8)
memory usage: 75.2+ MB


In [10]:
#convert all string vaoues to float
data_df['date'] = pd.to_datetime(data_df['date'], format='%Y/%m/%d')
data_df['station_mean_temperature_c'] = pd.to_numeric(data_df['station_mean_temperature_c'], errors='coerce')
data_df['station_air_temperature_c'] = pd.to_numeric(data_df['station_air_temperature_c'], errors='coerce')
data_df['Station_TDMEAN_c'] = pd.to_numeric(data_df['Station_TDMEAN_c'], errors='coerce')
data_df['Maximum_Sustained_Wind_Speed'] = pd.to_numeric(data_df['Maximum_Sustained_Wind_Speed'], errors='coerce')
data_df['Mean_Wind_Speed'] = pd.to_numeric(data_df['Mean_Wind_Speed'], errors='coerce')
data_df['Mean_Sea_Level_Pressure'] = pd.to_numeric(data_df['Mean_Sea_Level_Pressure'], errors='coerce')
data_df['Mean_Station_Pressure'] = pd.to_numeric(data_df['Mean_Station_Pressure'], errors='coerce')

In [11]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1095213 entries, 0 to 1095212
Data columns (total 9 columns):
 #   Column                        Non-Null Count    Dtype         
---  ------                        --------------    -----         
 0   station_number                1095213 non-null  int64         
 1   date                          1095213 non-null  datetime64[ns]
 2   station_mean_temperature_c    994114 non-null   float64       
 3   station_air_temperature_c     1049920 non-null  float64       
 4   Station_TDMEAN_c              1003964 non-null  float64       
 5   Maximum_Sustained_Wind_Speed  525380 non-null   float64       
 6   Mean_Wind_Speed               1032362 non-null  float64       
 7   Mean_Sea_Level_Pressure       791206 non-null   float64       
 8   Mean_Station_Pressure         793886 non-null   float64       
dtypes: datetime64[ns](1), float64(7), int64(1)
memory usage: 75.2 MB


In [12]:
data_df=data_df[['date','station_number', 'Station_TDMEAN_c', 'Mean_Wind_Speed',	'Mean_Sea_Level_Pressure', 'Mean_Station_Pressure', 'Maximum_Sustained_Wind_Speed']]

In [13]:
data_df.head()

,date,station_number,Station_TDMEAN_c,Mean_Wind_Speed,Mean_Sea_Level_Pressure,Mean_Station_Pressure,Maximum_Sustained_Wind_Speed
0,2001-01-01,1006,20.6,13.7,1007.5,1007.0,NaN
1,2001-01-02,1006,20.7,9.5,1008.3,1007.8,NaN
2,2001-01-03,1006,22.4,12.7,1007.8,1007.3,NaN
3,2001-01-04,1006,22.7,10.9,1006.8,1006.3,NaN
4,2001-01-05,1006,23.9,14.6,1007.0,1006.5,NaN


In [14]:
station_data_df=pd.merge(data_df, station_df, how="left", on=["station_number"])

In [15]:
station_data_df.head()

,date,station_number,Station_TDMEAN_c,Mean_Wind_Speed,Mean_Sea_Level_Pressure,Mean_Station_Pressure,Maximum_Sustained_Wind_Speed,Station_Name,Actual_open,Open,Close,LAT,LON,Height_m
0,2001-01-01,1006,20.6,13.7,1007.5,1007.0,NaN,WYNDHAM AERO,01/1951,1951-01-01,2021-01-31,-15.51,128.1503,3.8
1,2001-01-02,1006,20.7,9.5,1008.3,1007.8,NaN,WYNDHAM AERO,01/1951,1951-01-01,2021-01-31,-15.51,128.1503,3.8
2,2001-01-03,1006,22.4,12.7,1007.8,1007.3,NaN,WYNDHAM AERO,01/1951,1951-01-01,2021-01-31,-15.51,128.1503,3.8
3,2001-01-04,1006,22.7,10.9,1006.8,1006.3,NaN,WYNDHAM AERO,01/1951,1951-01-01,2021-01-31,-15.51,128.1503,3.8
4,2001-01-05,1006,23.9,14.6,1007.0,1006.5,NaN,WYNDHAM AERO,01/1951,1951-01-01,2021-01-31,-15.51,128.1503,3.8


In [16]:
station_data_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1095213 entries, 0 to 1095212
Data columns (total 14 columns):
 #   Column                        Non-Null Count    Dtype         
---  ------                        --------------    -----         
 0   date                          1095213 non-null  datetime64[ns]
 1   station_number                1095213 non-null  int64         
 2   Station_TDMEAN_c              1003964 non-null  float64       
 3   Mean_Wind_Speed               1032362 non-null  float64       
 4   Mean_Sea_Level_Pressure       791206 non-null   float64       
 5   Mean_Station_Pressure         793886 non-null   float64       
 6   Maximum_Sustained_Wind_Speed  525380 non-null   float64       
 7   Station_Name                  1095213 non-null  object        
 8   Actual_open                   1095213 non-null  object        
 9   Open                          1095213 non-null  datetime64[ns]
 10  Close                         1095213 non-null  datetime64[ns]
 11

In [17]:
#station_data_df.to_csv('WAWeatherStationDataSet.csv',index=False)
#files.download("/content/WAWeatherStationDataSet.csv")

In [18]:
ver6_df=pd.read_excel('https://drive.google.com/uc?id=1P1W68_Q-JnpW9LqIb_63-uZBM4RdZkin', header=0)

In [19]:
ver6_df.head()

,fih_fire_s,fih_year1,fih_season,fih_distri,fih_hist_d,fih_number,fih_fire_t,fih_date1,fih_cause,fih_ignit_,fih_detect,fih_capt_m,fih_author,fih_poly_t,fih_commen,fih_name,fih_perime,LAT,LON,ACRES,MODIS,NC_LAT,NC_LON,lfmc_mean,lfmc_stdv,quality_mask,ELEVATION,dis_latlon_n_nclatlon,climate_LAT,climate_LON,rain_fall_mm,max_temperature_c,min_temperature_c,vp_9am_hpa,vp_3pm_hpa,dis_latlon_n_climatelatlon,population,LGA
0,2000/2001,2001,SU,PHL,Mundaring,999,WF,2001-01-01,999,0,0,HIM,,Actual Burnt,,,9.653766,-31.3497,116.0366,850.9043,h27v12,-31.350992,116.036200,36.224998,18.932627,1.0,206,148.566183,-31.35,116.05,0.0,29.32,13.40,15.27,15.10,1272.918883,2948,Chittering (S)
1,2000/2001,2001,SU,PHL,Mundaring,999,WF,2001-01-01,999,0,0,HIM,,Actual Burnt,,,1.058290,-32.1413,116.1956,18.3947,h27v12,-32.142631,116.195939,61.924999,16.209669,1.0,266,151.398109,-32.15,116.20,0.0,29.39,14.58,14.62,14.69,1052.361097,52391,Armadale (C)
2,2000/2001,2001,SU,PHL,Mundaring,999,WF,2001-01-01,999,0,0,HIM,,Actual Burnt,,,0.157169,-32.0572,116.1932,0.4853,h27v12,-32.059300,116.193215,66.547501,13.480727,1.0,312,233.564838,-32.05,116.20,0.0,29.14,14.32,14.67,14.76,1025.507101,48743,Kalamunda (C)
3,2000/2001,2001,SU,PHL,Mundaring,999,WF,2001-01-01,999,0,0,HIM,,Actual Burnt,,,0.558105,-32.0542,116.1940,5.3271,h27v12,-32.055134,116.192840,61.877499,13.119061,1.0,311,150.803304,-32.05,116.20,0.0,29.14,14.32,14.67,14.76,733.392420,48743,Kalamunda (C)
4,2000/2001,2001,SU,PHL,Mundaring,999,WF,2001-01-01,999,0,0,HIM,,Actual Burnt,,,0.165064,-32.0476,116.1925,0.5353,h27v12,-32.046801,116.192091,83.957497,14.985291,1.0,309,96.865831,-32.05,116.20,0.0,29.14,14.32,14.67,14.76,755.563084,48743,Kalamunda (C)


In [20]:
#ver6_df['station_number']=np.nan
ver6_df['Station_TDMEAN_c']=np.nan
ver6_df['Mean_Wind_Speed']=np.nan
ver6_df['Mean_Sea_Level_Pressure']=np.nan
ver6_df['Mean_Station_Pressure']=np.nan
ver6_df['Maximum_Sustained_Wind_Speed']=np.nan
#ver6_df['delta']=np.nan

In [21]:
ver6_df.shape

(34932, 43)

In [22]:
uniq_dates=ver6_df['fih_date1'].drop_duplicates().reset_index().sort_values(by='fih_date1', ignore_index=True)
uniq_dates=uniq_dates.rename(columns={"fih_date1": "date"})

In [23]:
uniq_dates.head(1)

,index,date
0,0,2001-01-01


In [24]:
station_data_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1095213 entries, 0 to 1095212
Data columns (total 14 columns):
 #   Column                        Non-Null Count    Dtype         
---  ------                        --------------    -----         
 0   date                          1095213 non-null  datetime64[ns]
 1   station_number                1095213 non-null  int64         
 2   Station_TDMEAN_c              1003964 non-null  float64       
 3   Mean_Wind_Speed               1032362 non-null  float64       
 4   Mean_Sea_Level_Pressure       791206 non-null   float64       
 5   Mean_Station_Pressure         793886 non-null   float64       
 6   Maximum_Sustained_Wind_Speed  525380 non-null   float64       
 7   Station_Name                  1095213 non-null  object        
 8   Actual_open                   1095213 non-null  object        
 9   Open                          1095213 non-null  datetime64[ns]
 10  Close                         1095213 non-null  datetime64[ns]
 11

In [30]:
#Test
temp_df=station_data_df[(station_data_df['date']==uniq_dates['date'][0]) & (station_data_df['Mean_Wind_Speed'].notnull())].reset_index(drop=True)
OK = OrdinaryKriging(
      temp_df['LON'],
      temp_df['LAT'],
      temp_df['Mean_Wind_Speed'],
      variogram_model="linear",
      verbose=False,
      enable_plotting=False,
      coordinates_type="geographic",
)
temp2_df=ver6_df[ver6_df['fih_date1']==uniq_dates['date'][0]]
v,s2=OK.execute('points',temp2_df['LON'][0],temp2_df['LAT'][0])
v.data[0]

16.828878023572717

In [31]:
for i, day in uniq_dates.iterrows():
  if (i % 100)==0:
    print("Completed : "+str(round((i*100/uniq_dates.shape[0]),2))+"%")
  try:
    #Ordinary Kriging Station_TDMEAN_c
    #Here we use one day fo data as input for the kriging model and 5 parameter kriging
    #Station_TDMEAN_c
    #Get All station data related particular date
    input_df1=station_data_df[(station_data_df['date']==day['date']) & (station_data_df['Station_TDMEAN_c'].notnull())].reset_index(drop=True)
    OK_Station_TDMEAN_c = OrdinaryKriging(
          input_df1['LON'],
          input_df1['LAT'],
          input_df1['Station_TDMEAN_c'],
          variogram_model="linear",
          verbose=False,
          enable_plotting=False,
          coordinates_type="geographic",
    )
    #Mean_Wind_Speed
    input_df2=station_data_df[(station_data_df['date']==day['date']) & (station_data_df['Mean_Wind_Speed'].notnull())].reset_index(drop=True)
    OK_Mean_Wind_Speed = OrdinaryKriging(
          input_df2['LON'],
          input_df2['LAT'],
          input_df2['Mean_Wind_Speed'],
          variogram_model="linear",
          verbose=False,
          enable_plotting=False,
          coordinates_type="geographic",
    )
    #Mean_Sea_Level_Pressure
    input_df3=station_data_df[(station_data_df['date']==day['date']) & (station_data_df['Mean_Sea_Level_Pressure'].notnull())].reset_index(drop=True)
    OK_Mean_Sea_Level_Pressure = OrdinaryKriging(
          input_df3['LON'],
          input_df3['LAT'],
          input_df3['Mean_Sea_Level_Pressure'],
          variogram_model="linear",
          verbose=False,
          enable_plotting=False,
          coordinates_type="geographic",
    )
    #Mean_Station_Pressure.
    input_df4=station_data_df[(station_data_df['date']==day['date']) & (station_data_df['Mean_Station_Pressure'].notnull())].reset_index(drop=True)
    OK_Mean_Station_Pressure = OrdinaryKriging(
          input_df4['LON'],
          input_df4['LAT'],
          input_df4['Mean_Station_Pressure'],
          variogram_model="linear",
          verbose=False,
          enable_plotting=False,
          coordinates_type="geographic",
    )
    #Maximum_Sustained_Wind_Speed
    input_df5=station_data_df[(station_data_df['date']==day['date']) & (station_data_df['Maximum_Sustained_Wind_Speed'].notnull())].reset_index(drop=True)
    OK_Maximum_Sustained_Wind_Speed = OrdinaryKriging(
          input_df5['LON'],
          input_df5['LAT'],
          input_df5['Maximum_Sustained_Wind_Speed'],
          variogram_model="linear",
          verbose=False,
          enable_plotting=False,
          coordinates_type="geographic",
    )
    data_df=ver6_df[ver6_df['fih_date1']==day['date']]
    for j, row in data_df.iterrows():
      v1,s1=OK_Station_TDMEAN_c.execute('points',row['LON'],row['LAT'])
      ver6_df.at[j, 'Station_TDMEAN_c']=round(v1.data[0],1)

      v2,s2=OK_Mean_Wind_Speed.execute('points',row['LON'],row['LAT'])
      ver6_df.at[j, 'Mean_Wind_Speed']=round(v2.data[0],1)

      v3,s3=OK_Mean_Sea_Level_Pressure.execute('points',row['LON'],row['LAT'])
      ver6_df.at[j, 'Mean_Sea_Level_Pressure']=round(v3.data[0],1)

      v4,s4=OK_Mean_Station_Pressure.execute('points',row['LON'],row['LAT'])
      ver6_df.at[j, 'Mean_Station_Pressure']=round(v4.data[0],1)

      v5,s5=OK_Maximum_Sustained_Wind_Speed.execute('points',row['LON'],row['LAT'])
      ver6_df.at[j, 'Maximum_Sustained_Wind_Speed']=round(v5.data[0],1)
  except Exception as e:
    print(uniq_dates.loc[[i]])
    print("Kriging :"+str(e))
    print(traceback.format_exc())
print("Completed : 100.0%")

Completed : 0.0%
Completed : 2.78%
Completed : 5.56%
Completed : 8.34%
Completed : 11.11%
Completed : 13.89%
Completed : 16.67%
Completed : 19.45%
Completed : 22.23%
Completed : 25.01%
Completed : 27.79%
Completed : 30.56%
Completed : 33.34%
Completed : 36.12%
Completed : 38.9%
Completed : 41.68%
Completed : 44.46%
Completed : 47.24%
Completed : 50.01%
Completed : 52.79%
Completed : 55.57%
Completed : 58.35%
Completed : 61.13%
Completed : 63.91%
Completed : 66.69%
Completed : 69.46%
Completed : 72.24%
Completed : 75.02%
Completed : 77.8%
Completed : 80.58%
Completed : 83.36%
Completed : 86.14%
Completed : 88.91%
Completed : 91.69%
Completed : 94.47%
Completed : 97.25%
Completed : 100.0%


In [32]:
ver6_df.describe()

,fih_year1,fih_cause,fih_detect,fih_perime,LAT,LON,ACRES,NC_LAT,NC_LON,lfmc_mean,lfmc_stdv,quality_mask,ELEVATION,dis_latlon_n_nclatlon,climate_LAT,climate_LON,rain_fall_mm,max_temperature_c,min_temperature_c,vp_9am_hpa,vp_3pm_hpa,dis_latlon_n_climatelatlon,population,Station_TDMEAN_c,Mean_Wind_Speed,Mean_Sea_Level_Pressure,Mean_Station_Pressure,Maximum_Sustained_Wind_Speed
count,34932.000000,34932.000000,34932.000000,3.493200e+04,34932.000000,34932.000000,3.493200e+04,34932.000000,34932.000000,34932.000000,34932.000000,27678.0,34932.000000,34932.000000,34932.000000,34932.000000,34932.000000,34932.000000,34932.000000,34932.000000,34932.000000,34932.000000,34932.000000,3.493200e+04,3.493200e+04,34932.000000,34932.000000,34932.000000
mean,2010.638240,509.727700,22.575490,6.782555e+02,-28.016166,119.483834,2.579403e+03,-28.016176,119.483825,73.325162,16.603412,1.0,285.587828,178.310006,-28.016270,119.483784,0.986228,30.246669,15.217987,12.603731,11.232864,1999.312163,16040.583734,-1.014861e+14,-1.887749e+06,1014.032672,992.927700,43.269126
std,5.930556,498.081921,135.945735,2.558101e+04,4.829776,3.841658,3.077619e+04,4.829784,3.841657,39.655383,5.814712,0.0,153.217771,66.051728,4.830031,3.841724,3.478098,8.666026,6.788165,5.070869,5.153552,751.719408,33131.146595,9.766545e+15,2.353051e+08,5.832506,7.170177,9.131304
min,2001.000000,0.000000,0.000000,0.000000e+00,-35.112900,113.199200,0.000000e+00,-35.113361,113.197979,4.440000,3.458670,1.0,-17.000000,0.756998,-35.100000,113.200000,0.000000,10.960000,-1.460000,0.000000,0.000000,0.000000,79.000000,-1.146412e+18,-3.111168e+10,997.600000,962.600000,14.900000
25%,2005.000000,1.000000,0.000000,6.595387e-01,-32.749700,116.097575,4.935200e+00,-32.750943,116.097096,55.580002,13.143747,1.0,173.000000,131.838467,-32.750000,116.100000,0.000000,23.360000,9.520000,9.310000,7.400000,1465.839440,1216.000000,5.800000e+00,1.230000e+01,1009.700000,988.500000,38.600000
50%,2012.000000,999.000000,0.000000,2.115346e+00,-28.243750,118.087800,2.716345e+01,-28.242767,118.087562,57.397499,15.137430,1.0,282.000000,185.208259,-28.250000,118.100000,0.000000,30.860000,14.570000,12.030000,11.050000,2076.718856,7667.000000,1.000000e+01,1.470000e+01,1013.200000,994.300000,42.600000
75%,2016.000000,999.000000,0.000000,1.075653e+01,-22.896375,122.603875,2.462207e+02,-22.897119,122.603640,91.173750,18.194876,1.0,406.000000,227.984598,-22.900000,122.600000,0.100000,37.430000,20.520000,14.980000,14.000000,2566.608375,12881.000000,1.370000e+01,1.640000e+01,1017.800000,997.900000,46.600000
max,2020.000000,999.000000,2875.000000,3.636410e+06,-20.094700,128.999900,3.756446e+06,-20.093050,129.001360,299.887512,76.268379,1.0,979.000000,476.192770,-20.100000,129.000000,64.900000,48.670000,33.290000,38.840000,35.700000,3784.391761,208237.000000,5.914766e+16,8.976054e+09,1038.500000,1017.900000,90.200000


In [33]:
ver7_df=ver6_df.copy()
ver7_df['TMEAN_c']=ver6_df['max_temperature_c'].div(2)+ver6_df['min_temperature_c'].div(2)
ver7_df.describe()

,fih_year1,fih_cause,fih_detect,fih_perime,LAT,LON,ACRES,NC_LAT,NC_LON,lfmc_mean,lfmc_stdv,quality_mask,ELEVATION,dis_latlon_n_nclatlon,climate_LAT,climate_LON,rain_fall_mm,max_temperature_c,min_temperature_c,vp_9am_hpa,vp_3pm_hpa,dis_latlon_n_climatelatlon,population,Station_TDMEAN_c,Mean_Wind_Speed,Mean_Sea_Level_Pressure,Mean_Station_Pressure,Maximum_Sustained_Wind_Speed,TMEAN_c
count,34932.000000,34932.000000,34932.000000,3.493200e+04,34932.000000,34932.000000,3.493200e+04,34932.000000,34932.000000,34932.000000,34932.000000,27678.0,34932.000000,34932.000000,34932.000000,34932.000000,34932.000000,34932.000000,34932.000000,34932.000000,34932.000000,34932.000000,34932.000000,3.493200e+04,3.493200e+04,34932.000000,34932.000000,34932.000000,34932.000000
mean,2010.638240,509.727700,22.575490,6.782555e+02,-28.016166,119.483834,2.579403e+03,-28.016176,119.483825,73.325162,16.603412,1.0,285.587828,178.310006,-28.016270,119.483784,0.986228,30.246669,15.217987,12.603731,11.232864,1999.312163,16040.583734,-1.014861e+14,-1.887749e+06,1014.032672,992.927700,43.269126,22.732328
std,5.930556,498.081921,135.945735,2.558101e+04,4.829776,3.841658,3.077619e+04,4.829784,3.841657,39.655383,5.814712,0.0,153.217771,66.051728,4.830031,3.841724,3.478098,8.666026,6.788165,5.070869,5.153552,751.719408,33131.146595,9.766545e+15,2.353051e+08,5.832506,7.170177,9.131304,7.495050
min,2001.000000,0.000000,0.000000,0.000000e+00,-35.112900,113.199200,0.000000e+00,-35.113361,113.197979,4.440000,3.458670,1.0,-17.000000,0.756998,-35.100000,113.200000,0.000000,10.960000,-1.460000,0.000000,0.000000,0.000000,79.000000,-1.146412e+18,-3.111168e+10,997.600000,962.600000,14.900000,6.625000
25%,2005.000000,1.000000,0.000000,6.595387e-01,-32.749700,116.097575,4.935200e+00,-32.750943,116.097096,55.580002,13.143747,1.0,173.000000,131.838467,-32.750000,116.100000,0.000000,23.360000,9.520000,9.310000,7.400000,1465.839440,1216.000000,5.800000e+00,1.230000e+01,1009.700000,988.500000,38.600000,16.625000
50%,2012.000000,999.000000,0.000000,2.115346e+00,-28.243750,118.087800,2.716345e+01,-28.242767,118.087562,57.397499,15.137430,1.0,282.000000,185.208259,-28.250000,118.100000,0.000000,30.860000,14.570000,12.030000,11.050000,2076.718856,7667.000000,1.000000e+01,1.470000e+01,1013.200000,994.300000,42.600000,22.600000
75%,2016.000000,999.000000,0.000000,1.075653e+01,-22.896375,122.603875,2.462207e+02,-22.897119,122.603640,91.173750,18.194876,1.0,406.000000,227.984598,-22.900000,122.600000,0.100000,37.430000,20.520000,14.980000,14.000000,2566.608375,12881.000000,1.370000e+01,1.640000e+01,1017.800000,997.900000,46.600000,29.140000
max,2020.000000,999.000000,2875.000000,3.636410e+06,-20.094700,128.999900,3.756446e+06,-20.093050,129.001360,299.887512,76.268379,1.0,979.000000,476.192770,-20.100000,129.000000,64.900000,48.670000,33.290000,38.840000,35.700000,3784.391761,208237.000000,5.914766e+16,8.976054e+09,1038.500000,1017.900000,90.200000,39.580000


In [34]:
ver7_df.to_excel('WesternAustraliaFireData-ver7-Okrig.xlsx',index=False)
files.download("/content/WesternAustraliaFireData-ver7-Okrig.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>